In [1]:
import os
import numpy as np
import pandas as pd

# set seed for the reproducible result
SEED = 42

In [2]:
augmented_dir = './data/_augmented_features' # we will get names from the augmented proteins
augmented_proteins = [x[:-4] for x in os.listdir(augmented_dir) if x[-3:] == 'csv'] # get protein name list to be processed for building machine learning models
print('the number of initial proteins:', len(augmented_proteins))
print(augmented_proteins[:10])

the number of initial proteins: 121
['24622_2', 'A0A024RAY2_P05783', 'A2ABU4', 'A2AHJ4', 'A2AKB9', 'A2AQ25', 'E9K9Z1', 'E9Q1P8', 'E9Q5G3', 'O08537']


# build amino acid sequence dataset

In [3]:
oglcnac_data = pd.read_csv('./data\oglcnacome_sites.csv', index_col=0) 
print(oglcnac_data.info())
oglcnac_data.sample(10, random_state=SEED)

<class 'pandas.core.frame.DataFrame'>
Index: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   UniprotKB ID   4501 non-null   object 
 1   organism       4501 non-null   object 
 2   oglcnacscore   4501 non-null   float64
 3   oglcnac sites  4501 non-null   object 
 4   sequence       4501 non-null   object 
dtypes: float64(1), object(4)
memory usage: 211.0+ KB
None


,UniprotKB ID,organism,oglcnacscore,oglcnac sites,sequence
1634,Q00587,Homo sapiens,12.207032,[132],MPGPQGGRGAATMSLGKLSPVGWVSSSQGKRRLTADMISHPLGDFR...
3687,Q9BQC3,Homo sapiens,27.611586,"[467, 470, 474]",MESMFSSPAEAALQRETGVPGLLTPLPDLDGVYELERVAGFVRDLG...
4244,Q9UIJ7,Homo sapiens,11.665506,[18],MGASARLLRAVIMGAPGSGKGTVSSRITTHFELKHLSSGDLLRDNM...
4005,Q9NQ66,Homo sapiens,6.005927,"[187, 473]",MAGAQPGVHALQLKPVCVSDSLKKGTKFVKWDDDSTIVTPIILRTD...
468,O75152,Homo sapiens,5.077726,"[382, 443, 543, 579]",MPNQGEDCYFFFYSTCTKGDSCPFRHCEAAIGNETVCTLWQEGRCF...
3042,Q8N135,Homo sapiens,12.716548,[337],MGGAGILLLLLAGAGVVVAWRPPKGKCPLRCSCSKDSALCEGSPDL...
1910,Q14247,Homo sapiens,18.858372,"[11, 240, 277, 322, 323, 328, 331, 332, 345, 401]",MWKASAGHAVSIAQDDAGADDWETDPDFVNDVSEKEQRWGAKTVQG...
151,E9Q555,Mus musculus,11.752367,"[1202, 1206]",MECPQCGHVSSEKAPKFCSECGQKLPSAATVQGDLKNDNTLVVSST...
3896,Q9H4A4,Homo sapiens,16.163186,"[60, 247]",MASGEHSPGSGAARRPLHSAQAVDVASASNFRAFELLHLHLDLRAE...
2749,Q80TE4,Mus musculus,20.868470,"[1702, 1704]",MSDPRPSQAEKHKLGRAAAKLKDPSRTMQADDYFARKFKAINGSMG...


In [4]:
from functions import *

In [5]:
'''
augmented protein names include either single element like 'P53621' or multiple elements like 'A0A024RAY2_P05783'
first step is to check if name elements exist in the o-glcnacome database column 'UniprotKB ID' 
if any element exists in the database, then build a dataset of the protein's amino acid sequence with its positive locations
'''

# Convert UniprotKB IDs to a set for faster lookup
database_proteins = set(oglcnac_data['UniprotKB ID'].values)

# Initialize dictionaries and list
positivity_data = {}  # "protein name : sequence with positive sites"
name_augmented_oglcnacome = {}  # names between augmented proteins and database
not_in_database = []  # proteins not in the database

for protein_name in augmented_proteins:
    name_elements = protein_name.split('_')  # Split protein names by '_'
    
    for element in name_elements:
        if element in database_proteins:
            oglcnac_name = element
            break
        else:
            oglcnac_name = None
    
    if oglcnac_name: 
        # Update name matching if protein name has multiple elements
        if len(name_elements) > 1:
            name_augmented_oglcnacome[protein_name] = oglcnac_name
        
        # Retrieve sequence data and update dataset
        protein_oglcnac = oglcnac_data[oglcnac_data['UniprotKB ID'] == oglcnac_name]
        positivity_data[protein_name] = sequence_with_positivity(protein_oglcnac)
    else:
        not_in_database.append(protein_name)

print(f'total number of matching proteins: {len(positivity_data)}')

total number of matching proteins: 112


In [6]:
print('augmented proteins : o-glcnacome database')
display(name_augmented_oglcnacome)

augmented proteins : o-glcnacome database


{'A0A024RAY2_P05783': 'P05783',
 'P0CG62_P0CG49': 'P0CG49',
 'P24622_2': 'P24622',
 'P63249_P63248': 'P63248',
 'P68406_P24622_2': 'P24622',
 'Q4R561_P60710': 'P60710',
 'Q9WVB1_P35279': 'P35279'}

In [7]:
print('proteins not in o-glcnacome database')
print(not_in_database)

proteins not in o-glcnacome database
['24622_2', 'E9K9Z1', 'O08984', 'P02470', 'P02488', 'P02505', 'P04799', 'P05451', 'P07756']


## set variables

In [8]:
for_onehot = { # column_name : classes
    # for input variables
    'residue' : ['A', 'R', 'N', 'D', 'C',
                 'E', 'Q', 'G', 'H', 'I',
                 'L', 'K', 'M', 'F', 'P',
                 'S', 'T', 'W', 'Y', 'V'],
    
    # for output variables
    'positivity' : [0, 1]
}

x_cts = []
x_cat = ['residue']
x_var = x_cts + x_cat

y_cts = []
y_cat = ['positivity']
y_var = y_cts + y_cat

## hyper parameter optimization by K-fold cross-validation

In [9]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold

import time
from IPython.display import clear_output

from ml_models import *

epochs = 1000
from keras.callbacks import EarlyStopping
patience = 30
callbacks = [EarlyStopping(patience=patience, restore_best_weights=True, monitor='val_loss')]

test_size = 0.2


### set initial parameters

In [10]:
model_type = 'LSTM_UP'

initial_params = default_params.copy()
initial_params.update({
    'window_size'    : 10
    })

print('initial parameters')
for key, value in initial_params.items():
    print(f'{key:<14} : {value}')

search_space = {
    'rnn_layers'    : [1, 2, 3, 4, 5],
    'rnn_neurons'   : [32, 64, 128, 256],
    'dnn_layers'    : [1, 2, 3, 4, 5],
    'dnn_neurons'   : [32, 64, 128, 256],
    'learning_rate' : [0.0001, 0.001, 0.01]
}

initial parameters
rnn_layers     : 1
rnn_neurons    : 64
dnn_layers     : 3
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


### model training with K-fold

In [11]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update  = False

params = initial_params.copy()

MODELs = []
METRICs = []
METRIC_MEAN = []
model_id = 1
verbose = 0
for param_name, space in search_space.items():
    for point in space:
        clear_output(wait=True)
        display(METRIC_MEAN)
        params[param_name] = point
        for key, value in search_space.items():
            print(f'{key:<14} : {value}')
        
        data_x = []
        data_y = []
        for protein_name in positivity_data:
            positivity = positivity_data.get(protein_name)
            ST_idx = np.where((positivity['residue'] == 'S') | (positivity['residue'] == 'T'))[0]
            
            # get X dataset
            x_onehot = get_onehots(positivity[x_var], columns = x_cat, for_onehot = for_onehot)
            x_features = list(x_onehot.columns)
            
            # get Y dataset
            y_onehot = get_onehots(positivity[y_var], columns = y_cat, for_onehot = for_onehot)
            y_labels = list(y_onehot.columns)
            
            for idx in ST_idx:
                window_x = np.array(get_window(x_onehot, idx, params['window_size']))
                label_y  = np.array(y_onehot.iloc[idx])
                
                data_x.append(window_x)
                data_y.append(label_y)
                
        data_x = np.array(data_x)
        data_y = np.array(data_y)
        model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) # I don't know why, but this row is helping producing the same training result of a neural network

        print('data x shape:', data_x.shape)
        print('data y shape:', data_y.shape)
        print('class y counts:', data_y.sum(0))
        print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')
        
        splitter = StratifiedShuffleSplit(n_splits = 1, test_size = test_size, random_state = SEED)
        train_idx, test_idx = list(splitter.split(data_x, data_y))[0]
        
        train_x = data_x[train_idx]
        train_y = data_y[train_idx]
        
        test_x = data_x[test_idx]
        test_y = data_y[test_idx]
        
        splitter_kf = KFold(n_splits = 5)
        for cv_idx, (train_idx_kf, test_idx_kf) in enumerate(splitter_kf.split(train_x, train_y)):
            train_x_kf, train_y_kf = train_x[train_idx_kf], train_y[train_idx_kf]
            train_x_kf, train_y_kf = upsample_data(train_x_kf, train_y_kf) # up-sample the training dataset
            test_x_kf, test_y_kf = train_x[test_idx_kf], train_y[test_idx_kf]
            
            model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) 
            model_name = name_model(f'{model_type}_KFOLD', params)
            
            
            model_folder  = f'./models/{model_name}_{train_x.shape}_{test_x.shape}'
            if not os.path.exists(model_folder):
                os.makedirs(model_folder)
            model_path    = f'{model_folder}/{cv_idx}.h5'
            metric_path   = f'{model_folder}/{cv_idx}.csv'
            
            
            if not os.path.exists(model_path) or model_update:
                time_start = time.time()
                history = model.fit(train_x_kf, train_y_kf, verbose=verbose, 
                                    epochs = 10000, callbacks = callbacks,
                                    validation_data = (test_x_kf, test_y_kf))
                time_end = time.time()
                training_time = round((time_end - time_start)/60, 3)
                
                model.save_weights(model_path)
                
                test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
                model_metrics = {
                    'model_id' : model_id,
                    'cv_idx'   : cv_idx,
                    **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
                    'train_y'     : train_y.shape[-1],
                    'test_size'   : test_x.shape[0],
                    **params,
                    'regularizer_input' : params['regularizer']['input'],
                    'regularizer_hidden' : params['regularizer']['hidden'],
                    'regularizer_bias' : params['regularizer']['bias'],
                    'training_time': training_time,
                    'test_loss': test_loss,
                    'accuracy': accuracy,
                    **{f'precision_{x}': precision[x] for x in range(len(precision))},
                    **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
                    **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
                
                model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
                model_metrics.to_csv(metric_path, index=False)
                
            else:
                model.load_weights(model_path)
                model_metrics = pd.read_csv(metric_path, header=0)
                
            print(f'f1 score: {model_metrics.f1_1[0]}')
            
            model_metrics['model_id'] = model_id
            METRICs.append(model_metrics)
            MODELs.append(model)
        
        METRIC_MEAN = pd.concat(METRICs).groupby('model_id').mean()
        f1_best = METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0].f1_1
        print(f'best f1 score: {f1_best}')
        model_id += 1
    if param_name in ['learning_rate']: # for float-type parameters
        best_value = float(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"
    else: # for int-type parameters
        best_value = int(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"

clear_output(wait=True)
display(METRIC_MEAN)
for key, value in search_space.items():
    print(f'{key:<14} : {value}')


,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,9696.0,21.0,20.0,2.0,2424.0,1.0,64.0,3.0,64.0,...,NaN,1.1396,0.387555,85.610,97.438,11.064,87.352,40.442,92.114,17.346
2,2.0,9696.0,21.0,20.0,2.0,2424.0,2.0,64.0,3.0,64.0,...,NaN,2.0034,0.375400,90.214,97.626,18.908,92.084,41.778,94.732,25.036
3,2.0,9696.0,21.0,20.0,2.0,2424.0,3.0,64.0,3.0,64.0,...,NaN,5.9026,0.264431,94.374,97.410,29.512,96.726,33.332,97.066,30.976
4,2.0,9696.0,21.0,20.0,2.0,2424.0,4.0,64.0,3.0,64.0,...,NaN,5.9872,0.226814,94.752,97.464,31.612,97.078,34.444,97.268,32.868
5,2.0,9696.0,21.0,20.0,2.0,2424.0,5.0,64.0,3.0,64.0,...,NaN,5.1272,0.235868,93.672,97.534,27.658,95.854,37.108,96.674,31.060
6,2.0,9696.0,21.0,20.0,2.0,2424.0,4.0,32.0,3.0,64.0,...,NaN,6.4446,0.307331,90.874,97.530,26.662,92.886,38.666,94.970,28.646
7,2.0,9696.0,21.0,20.0,2.0,2424.0,4.0,64.0,3.0,64.0,...,NaN,5.9872,0.226814,94.752,97.464,31.612,97.078,34.444,97.268,32.868
8,2.0,9696.0,21.0,20.0,2.0,2424.0,4.0,128.0,3.0,64.0,...,NaN,5.1034,0.220633,95.180,97.450,35.748,97.550,33.778,97.496,34.316
9,2.0,9696.0,21.0,20.0,2.0,2424.0,4.0,256.0,3.0,64.0,...,NaN,4.7260,0.216315,95.040,97.390,34.008,97.464,32.222,97.424,32.658


rnn_layers     : [1, 2, 3, '4', 5]
rnn_neurons    : [32, 64, '128', 256]
dnn_layers     : [1, 2, 3, 4, '5']
dnn_neurons    : [32, '64', 128, 256]
learning_rate  : [0.0001, '0.001', 0.01]


In [12]:
print('data x shape: ', data_x.shape)
print('data y shape: ', data_y.shape)
print('train x shape:', train_x.shape)
print('test  x shape:', test_y.shape)

data x shape:  (12120, 21, 20)
data y shape:  (12120, 2)
train x shape: (9696, 21, 20)
test  x shape: (2424, 2)


In [13]:
print('optimal parameters')
for key, value in params.items():
    print(f'{key:<14} : {value}')

optimal parameters
rnn_layers     : 4
rnn_neurons    : 128
dnn_layers     : 5
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## evaluate model's general performance through Monte-Carlo cross-validation

In [14]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update = False
            
MODELs = []
METRICs = []
METRIC_MEAN = []
verbose = 0
        
data_x = []
data_y = []

for protein_name in positivity_data:
    positivity = positivity_data.get(protein_name)
    ST_idx = np.where((positivity['residue'] == 'S') | (positivity['residue'] == 'T'))[0]
    
    # get X dataset
    x_onehot = get_onehots(positivity[x_var], columns = x_cat, for_onehot = for_onehot)
    x_features = list(x_onehot.columns)
    
    # get Y dataset
    y_onehot = get_onehots(positivity[y_var], columns = y_cat, for_onehot = for_onehot)
    y_labels = list(y_onehot.columns)
    
    for idx in ST_idx:
        window_x = np.array(get_window(x_onehot, idx, params['window_size']))
        label_y  = np.array(y_onehot.iloc[idx])
        
        data_x.append(window_x)
        data_y.append(label_y)
        
data_x = np.array(data_x)
data_y = np.array(data_y)

print('data x shape:', data_x.shape)
print('data y shape:', data_y.shape)
print('class y counts:', data_y.sum(0))
print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')

splitter = StratifiedShuffleSplit(n_splits = 5, test_size = test_size, random_state = SEED)
for cv_idx, (train_idx, test_idx) in enumerate(splitter.split(data_x, data_y)):
    train_x, train_y = data_x[train_idx], data_y[train_idx]
    train_x, train_y = upsample_data(train_x, train_y) # up-sample the training dataset
    test_x , test_y  = data_x[test_idx],  data_y[test_idx]
    
    model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params)
    model_name = name_model(f'{model_type}', params)
    
    
    model_folder  = f'./models/{model_name}_{train_x.shape}_{test_x.shape}'
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    model_path    = f'{model_folder}/{cv_idx}.h5'
    metric_path   = f'{model_folder}/{cv_idx}.csv'
    
    
    if not os.path.exists(model_path) or model_update:
        time_start = time.time()
        history = model.fit(train_x, train_y, verbose=verbose, 
                            epochs = 10000, callbacks = callbacks,
                            validation_split = test_size/(1-test_size))
        time_end = time.time()
        training_time = round((time_end - time_start)/60, 3)
        
        model.save_weights(model_path)
        
        test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
        model_metrics = {
            'model_id' : model_id,
            'cv_idx'   : cv_idx,
            **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
            'train_y'     : train_y.shape[-1],
            'test_size'   : test_x.shape[0],
            **params,
            'regularizer_input' : params['regularizer']['input'],
            'regularizer_hidden' : params['regularizer']['hidden'],
            'regularizer_bias' : params['regularizer']['bias'],
            'training_time': training_time,
            'test_loss': test_loss,
            'accuracy': accuracy,
            **{f'precision_{x}': precision[x] for x in range(len(precision))},
            **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
            **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
        model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
        model_metrics.to_csv(metric_path, index=False)
        
    else:
        model.load_weights(model_path)
        model_metrics = pd.read_csv(metric_path, header=0)
        
    print(f'f1 score: {model_metrics.f1_1[0]}')
    
    METRICs.append(model_metrics)
    MODELs.append(model)

METRICs = pd.concat(METRICs)
METRIC_MEAN = METRICs.groupby('model_id').mean()
METRIC_STD = METRICs.groupby('model_id').std()

data x shape: (12120, 21, 20)
data y shape: (12120, 2)
class y counts: [11672   448]
class y ratio: [0.963 0.037]
f1 score: 39.290000000000006
f1 score: 24.49
f1 score: 31.65
f1 score: 30.97
f1 score: 27.17


In [15]:
METRICs

,model_id,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
0,22,0,18676,21,20,2,2424,4,128,5,...,NaN,8.125,0.246036,95.79,97.57,42.31,98.07,36.67,97.82,39.29
0,22,1,18676,21,20,2,2424,4,128,5,...,NaN,9.376,0.377182,95.42,96.96,31.58,98.33,20.00,97.64,24.49
0,22,2,18676,21,20,2,2424,4,128,5,...,NaN,7.977,0.257894,95.54,97.24,36.76,98.16,27.78,97.70,31.65
0,22,3,18676,21,20,2,2424,4,128,5,...,NaN,13.241,0.347051,95.59,97.20,36.92,98.24,26.67,97.72,30.97
0,22,4,18676,21,20,2,2424,4,128,5,...,NaN,5.964,0.247780,94.47,97.21,26.60,97.04,27.78,97.13,27.17


In [16]:
METRIC_MEAN

,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
22,2.0,18676.0,21.0,20.0,2.0,2424.0,4.0,128.0,5.0,64.0,...,NaN,8.9366,0.295188,95.362,97.236,34.834,97.968,27.78,97.602,30.714


In [17]:
METRIC_STD

,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
22,1.581139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2.698926,0.062183,0.516207,0.217784,5.96529,0.527608,5.932803,0.271698,5.607752
